# 12.02 - BeautifulSoup: Parsear HTML

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 30 min

---

## ¿Qué aprenderás?

- Entender la estructura del DOM HTML
- Parsear HTML con BeautifulSoup
- Seleccionar elementos con CSS selectors y métodos `.find()`
- Extraer texto, atributos y tablas
- Combinar `requests` + `BeautifulSoup` en un flujo completo

---

## 1. ¿Qué es el DOM?

El HTML de una página es un árbol de nodos (`tags`). BeautifulSoup nos permite navegarlo como si fuera Python.

```html
<div class="station">
  <h2>Sol</h2>
  <span class="bikes">12 bicis</span>
</div>
```

In [ ]:
# pip install beautifulsoup4 lxml
from bs4 import BeautifulSoup
import requests

# HTML de ejemplo (simulando una página de estaciones)
html = """
<html>
<body>
  <h1>Estaciones BiciMAD</h1>
  <div class="station" id="s1">
    <h2 class="name">Sol</h2>
    <span class="bikes">12</span>
    <span class="slots">8</span>
  </div>
  <div class="station" id="s2">
    <h2 class="name">Atocha</h2>
    <span class="bikes">5</span>
    <span class="slots">15</span>
  </div>
  <div class="station" id="s3">
    <h2 class="name">Cibeles</h2>
    <span class="bikes">0</span>
    <span class="slots">20</span>
  </div>
</body>
</html>
"""

soup = BeautifulSoup(html, 'lxml')
print(type(soup))
print(soup.title)  # None porque no hay <title>

---

## 2. Navegación básica

In [ ]:
# find() → primer elemento que coincide
titulo = soup.find('h1')
print(f"Título: {titulo.text}")

# find_all() → lista de todos los que coinciden
stations = soup.find_all('div', class_='station')
print(f"Estaciones encontradas: {len(stations)}")

In [ ]:
# Acceder a atributos
for station in stations:
    station_id = station['id']
    name = station.find('h2', class_='name').text
    bikes = station.find('span', class_='bikes').text
    slots = station.find('span', class_='slots').text
    print(f"{station_id}: {name} | Bicis: {bikes} | Huecos: {slots}")

---

## 3. CSS Selectors con select()

In [ ]:
# select() usa sintaxis CSS → más potente para estructuras complejas

# Todos los h2 dentro de .station
names = soup.select('div.station h2.name')
print("Nombres:", [n.text for n in names])

# Elemento con ID específico
sol = soup.select_one('#s1')
print(f"Estación Sol: {sol.find('h2').text}")

# Solo estaciones con bicis disponibles (no es posible con CSS puro,
# lo hacemos en Python después de extraer)
available = [
    s for s in soup.select('div.station')
    if int(s.find('span', class_='bikes').text) > 0
]
print(f"\nEstaciones con bicis: {len(available)}")
for s in available:
    print(f"  {s.find('h2').text}")

---

## 4. Extraer tablas HTML

In [ ]:
import pandas as pd

html_tabla = """
<table>
  <thead>
    <tr><th>Estación</th><th>Bicis</th><th>Huecos</th><th>Distrito</th></tr>
  </thead>
  <tbody>
    <tr><td>Sol</td><td>12</td><td>8</td><td>Centro</td></tr>
    <tr><td>Atocha</td><td>5</td><td>15</td><td>Arganzuela</td></tr>
    <tr><td>Cibeles</td><td>0</td><td>20</td><td>Centro</td></tr>
    <tr><td>Retiro</td><td>18</td><td>2</td><td>Retiro</td></tr>
  </tbody>
</table>
"""

# pandas.read_html() extrae tablas directamente
df = pd.read_html(html_tabla)[0]
print(df)
print(f"\nEstaciones con bicis disponibles:")
print(df[df['Bicis'] > 0])

---

## 5. Flujo completo: requests + BeautifulSoup

In [ ]:
def scrape_wikipedia_table(url: str, table_index: int = 0) -> pd.DataFrame | None:
    """
    Extrae una tabla de Wikipedia y la devuelve como DataFrame.

    Parameters
    ----------
    url : str
        URL de la página de Wikipedia
    table_index : int
        Índice de la tabla a extraer (0 = primera)

    Returns
    -------
    pd.DataFrame | None
        Tabla extraída o None si hay error
    """
    headers = {'User-Agent': 'DataPortfolio/1.0 (educational)'}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        tables = pd.read_html(response.text)
        if table_index >= len(tables):
            print(f"Solo hay {len(tables)} tablas en la página")
            return None

        return tables[table_index]

    except requests.exceptions.RequestException as e:
        print(f"Error de red: {e}")
        return None


# Ejemplo: sistemas de bicicletas compartidas en Wikipedia
url = "https://es.wikipedia.org/wiki/Bicicleta_compartida"
df = scrape_wikipedia_table(url, table_index=0)

if df is not None:
    print(f"Filas: {len(df)} | Columnas: {list(df.columns)}")
    print(df.head())

---

## Resumen

| Método | Uso |
|---|---|
| `BeautifulSoup(html, 'lxml')` | Parsear HTML |
| `soup.find('tag', class_='x')` | Primer elemento |
| `soup.find_all('tag')` | Todos los elementos |
| `soup.select('div.clase h2')` | CSS selector |
| `element.text` | Texto del nodo |
| `element['atributo']` | Atributo HTML |
| `pd.read_html(html)` | Tablas directamente |

---

## Ejercicio

Extrae la tabla de los sistemas de bikesharing más grandes del mundo de Wikipedia y crea un DataFrame con las columnas: `Ciudad`, `País`, `Bicis`. Ordénalo por número de bicis de mayor a menor.

In [ ]:
# Tu solución aquí

---

**Anterior:** [12.01 - Requests Basics](./12_01_requests_basics.ipynb)  
**Siguiente:** [12.03 - Scraping Real Cases](./12_03_scraping_real_cases.ipynb)